**Install MPI and mpi4py**

In [2]:
!apt-get update
!apt-get install -y mpich
!pip install mpi4py


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,708 kB]
Get:10 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [62.6 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,299 kB]
Get:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [

**Create Your MPI Python File**

In [3]:
%%writefile distributed_program.py

"""
Distributed Message-Passing Program with Unique Tasks
-----------------------------------------------------
Master process (rank 0):
    - Receives processed data from worker processes using comm.recv()
    - Displays the results along with the worker's rank

Worker processes (rank > 0):
    - Each worker has a unique list of 10 numbers based on its rank
    - Computes the product of its list
    - Sends both the data and the computed product to the master using comm.send()
"""

from mpi4py import MPI

# Initialize MPI communicator
comm = MPI.COMM_WORLD
rank = comm.Get_rank()   # Unique ID of each process
size = comm.Get_size()   # Total number of processes

# ---------------- MASTER PROCESS ----------------
if rank == 0:
    print("Master process started.\n")
    print(f"Total processes: {size}\n")

    # Receive results from all worker processes
    for worker_rank in range(1, size):
        # Receive message from the current worker
        data = comm.recv(source=worker_rank)
        print(f"Master received processed data from worker process {data['rank']}:")
        print(f"  Input data: {data['input_values']}")
        print(f"  Calculated product: {data['product']}\n")

# ---------------- WORKER PROCESSES ----------------
else:
    # Each worker gets a unique list of 10 numbers based on its rank
    start = (rank - 1) * 10 + 1
    end = start + 10   # exclusive
    numbers = list(range(start, end))

    # Compute the product of the numbers
    total_product = 1
    for num in numbers:
        total_product *= num

    # Prepare a message containing rank, task, and result
    message = {
        "rank": rank,
        "input_values": numbers,
        "product": total_product
    }

    # Send message to master process (rank 0)
    comm.send(message, dest=0)

    # Worker-side print removed for cleaner output


Overwriting distributed_program.py


**Run the MPI Program**

In [5]:
!mpiexec --allow-run-as-root --oversubscribe -n  python distributed_program.py


Master process started.

Total processes: 4

Master received processed data from worker process 1:
  Input data: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  Calculated product: 3628800

Master received processed data from worker process 2:
  Input data: [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
  Calculated product: 670442572800

Master received processed data from worker process 3:
  Input data: [21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
  Calculated product: 109027350432000



**PART III – DISCUSSION & REFLECTION**

**1. Why is message passing required in distributed systems?**

- In distributed systems, processes run on different nodes or separate parts of a system where they do not share the same RAM. Since one process cannot directly read the variables or data held by another, they need a formal way to communicate. Message passing acts like a postal service for data, allowing a worker process to package its results and send them to the master process. Without this, there would be no way to combine the work done by different processors into a single final result.

**2. What happens if one process fails?**

- In a standard MPI setup, the system is quite fragile when it comes to errors. If a worker process crashes or exits early, it fails to send the expected data back to the master. Because the master process uses a blocking receive command, it will sit and wait forever for that specific message to arrive, causing the entire program to hang or freeze. This demonstrates that basic message passing doesn't have built-in safety nets or "self-healing" features, so if one part fails, the whole job usually fails too.

**3. How does this model differ from shared-memory programming?**

- The main difference is in how the processes "talk" to each other and handle data. In shared-memory programming, all parts of the program can access the same memory space directly, which is fast but can lead to messy data conflicts. In the message-passing model, every process is isolated with its own private memory. If they need to share something, they must explicitly use "send" and "receive" functions. This makes the code a bit more organized and safer from accidental data overwrites, but it requires more careful planning to move data around.